In [1]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory

store = {}
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [2]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are a robot that classifies customer input messages into specific types. If order_id is provided, you need to review the previous conversation and output either '주문 변경' or '주문 취소'.
            If order_id is not provided, you need to classify the customer input message into one of the following two types:
            - 상품 문의, 주문 내역 조회, 주문 변경 내역 조회, 주문 취소 내역 조회: '문의'
            - 주문 요청, 주문 변경 요청, 주문 취소 요청: '요청'
            """
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "input:{input}\norder_id:{order_id}"),
    ]
)

In [3]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI()

In [4]:
chain = prompt | model

In [5]:
from langchain_core.runnables.history import RunnableWithMessageHistory

chain_with_memory = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [7]:
response = chain_with_memory.invoke(
    {"input": "떡케익5 얼마인가요", 
    "order_id": None},
    config={"configurable": {"session_id": "test_240515-1"}}
    )
response

Parent run aa06f4b1-1ea9-4f2d-9140-95c454ff8c1d not found for run af7ad9da-6a66-42de-9be3-3226317c7fa4. Treating as a root run.


AIMessage(content='문의', response_metadata={'token_usage': {'completion_tokens': 2, 'prompt_tokens': 181, 'total_tokens': 183}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-f4bb4fee-8de1-46a4-9c9e-3005cd9c943f-0')

In [8]:
response = chain_with_memory.invoke(
    {"input": "떡케익5 주문할게요", 
    "order_id": None},
    config={"configurable": {"session_id": "test_240515-1"}}
    )
response

Parent run 457b5769-16f7-4777-abd3-c81e784f6b5e not found for run 2055f270-23c3-4bda-bdf5-d72eb8f409b6. Treating as a root run.


AIMessage(content='요청', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 205, 'total_tokens': 208}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-b0ee7826-5bd8-43a6-836c-d0327662de95-0')

In [9]:
response = chain_with_memory.invoke(
    {"input": "떡케익5 취소할게요", 
    "order_id": None},
    config={"configurable": {"session_id": "test_240515-1"}}
    )
response

Parent run c2e8dbf4-60ee-45e1-a5f9-c33df5a33c7e not found for run eba03733-5f16-4da1-848d-babe2600bf98. Treating as a root run.


AIMessage(content='문의', response_metadata={'token_usage': {'completion_tokens': 2, 'prompt_tokens': 231, 'total_tokens': 233}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-439b0d12-01ad-493e-9d3c-979c2c4be83d-0')

In [10]:
response = chain_with_memory.invoke(
    {"input": "떡케익5 취소할게요", 
    "order_id": 3},
    config={"configurable": {"session_id": "test_240515-1"}}
    )
response

Parent run 3a7f890b-7d89-4498-b1c9-6017f9437558 not found for run e3dde834-0ea9-413f-a125-50185bfd31bf. Treating as a root run.


AIMessage(content='주문 취소', response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 256, 'total_tokens': 261}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-ef852833-394b-45f3-82a7-9a8fafdf09e8-0')

In [17]:
store["test_240515-1"].messages

[HumanMessage(content=''),
 AIMessage(content='문의', response_metadata={'token_usage': {'completion_tokens': 2, 'prompt_tokens': 154, 'total_tokens': 156}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-144a8985-6efb-4d9c-91bd-1e2016ec3683-0'),
 HumanMessage(content='떡케익5 얼마인가요'),
 AIMessage(content='문의', response_metadata={'token_usage': {'completion_tokens': 2, 'prompt_tokens': 181, 'total_tokens': 183}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-f4bb4fee-8de1-46a4-9c9e-3005cd9c943f-0'),
 HumanMessage(content='떡케익5 주문할게요'),
 AIMessage(content='요청', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 205, 'total_tokens': 208}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-b0ee7826-5bd8-43a6-836c-d0327662de95-0'),
 HumanMessage(content='떡케익5 취소할게요'),
 AIMessage(content=